## Uwekaji wa Neno

Katika mfano wetu wa awali, tulifanya kazi na vekta za maneno zenye vipimo vingi zenye urefu `vocab_size`, na tulibadilisha vekta za uwakilishi wa nafasi zenye vipimo vichache kuwa uwakilishi wa sparse one-hot. Uwiano huu wa one-hot hauhifadhi kumbukumbu kwa ufanisi. Zaidi ya hayo, kila neno linachukuliwa kuwa huru kutoka kwa mengine, hivyo vekta zilizowakilishwa kwa one-hot hazionyeshi uhusiano wa maana kati ya maneno.

Katika sehemu hii, tutaendelea kuchunguza seti ya data ya **News AG**. Kuanza, hebu tuweke data na tupate baadhi ya ufafanuzi kutoka sehemu iliyopita.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Embedding ni nini?

Wazo la **embedding** ni kuwakilisha maneno kwa kutumia vekta zenye dimensheni ndogo ambazo zinaonyesha maana ya kisemantiki ya neno. Tutajadili baadaye jinsi ya kujenga embeddings za maneno zenye maana, lakini kwa sasa wacha tuwaze embeddings kama njia ya kupunguza dimensheni ya vekta ya neno.

Kwa hivyo, safu ya embedding inachukua neno kama ingizo, na kutoa vekta ya matokeo yenye `embedding_size` maalum. Kwa namna fulani, ni sawa na safu ya `Dense`, lakini badala ya kuchukua vekta iliyosimbwa kwa one-hot kama ingizo, inaweza kuchukua nambari ya neno.

Kwa kutumia safu ya embedding kama safu ya kwanza katika mtandao wetu, tunaweza kubadilisha kutoka bag-of-words kwenda kwenye modeli ya **embedding bag**, ambapo tunabadilisha kila neno katika maandishi yetu kuwa embedding inayolingana, kisha tunahesabu kazi fulani ya jumla juu ya embeddings hizo zote, kama `sum`, `average` au `max`.

![Picha inayoonyesha classifier ya embedding kwa maneno matano ya mfululizo.](../../../../../translated_images/sw/embedding-classifier-example.b77f021a7ee67eee.webp)

Mtandao wetu wa neural classifier unajumuisha safu zifuatazo:

* Safu ya `TextVectorization`, ambayo inachukua kamba ya maandishi kama ingizo, na kutoa tensor ya nambari za tokeni. Tutataja ukubwa wa msamiati `vocab_size` unaofaa, na kupuuza maneno yanayotumika mara chache. Umbo la ingizo litakuwa 1, na umbo la matokeo litakuwa $n$, kwa kuwa tutapata tokeni $n$ kama matokeo, kila moja ikiwa na nambari kutoka 0 hadi `vocab_size`.
* Safu ya `Embedding`, ambayo inachukua nambari $n$, na kupunguza kila nambari kuwa vekta yenye densiti ya urefu fulani (100 katika mfano wetu). Kwa hivyo, tensor ya ingizo yenye umbo $n$ itabadilishwa kuwa tensor ya $n\times 100$. 
* Safu ya mkusanyiko, ambayo inachukua wastani wa tensor hii kwenye mhimili wa kwanza, yaani itahesabu wastani wa tensors zote $n$ za ingizo zinazolingana na maneno tofauti. Ili kutekeleza safu hii, tutatumia safu ya `Lambda`, na kupitisha ndani yake kazi ya kuhesabu wastani. Matokeo yatakuwa na umbo la 100, na litakuwa uwakilishi wa nambari wa mfululizo mzima wa ingizo.
* Mwisho, classifier ya `Dense` ya mstari.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Katika `muhtasari` wa matokeo, kwenye safu ya **umbo la matokeo**, kipimo cha kwanza cha tensor `None` kinahusiana na ukubwa wa minibatch, na cha pili kinahusiana na urefu wa mlolongo wa tokeni. Mlolongo wote wa tokeni kwenye minibatch una urefu tofauti. Tutajadili jinsi ya kushughulikia hili katika sehemu inayofuata.

Sasa hebu tufunze mtandao:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Kumbuka** kwamba tunajenga vectorizer kulingana na sehemu ndogo ya data. Hii inafanywa ili kuharakisha mchakato, na inaweza kusababisha hali ambapo si alama zote kutoka kwa maandishi yetu zipo kwenye msamiati. Katika hali hii, alama hizo zitaachwa, jambo ambalo linaweza kusababisha usahihi kidogo kupungua. Hata hivyo, katika maisha halisi sehemu ndogo ya maandishi mara nyingi hutoa makadirio mazuri ya msamiati.


### Kushughulikia ukubwa tofauti wa mlolongo wa vigezo

Tuelewe jinsi mafunzo yanavyofanyika katika minibatches. Katika mfano hapo juu, tensor ya pembejeo ina kipimo cha 1, na tunatumia minibatches zenye urefu wa 128, hivyo ukubwa halisi wa tensor ni $128 \times 1$. Hata hivyo, idadi ya tokeni katika kila sentensi ni tofauti. Tukitumia safu ya `TextVectorization` kwa pembejeo moja, idadi ya tokeni zinazorejeshwa ni tofauti, kulingana na jinsi maandishi yanavyogawanywa:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Hata hivyo, tunapotumia vectorizer kwa mfuatano kadhaa, inapaswa kutoa tensor ya umbo la mstatili, hivyo inajaza vipengele visivyotumika na tokeni ya PAD (ambayo katika kesi yetu ni sifuri):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Hapa tunaweza kuona viambatisho:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Kumbuka**: Ili kupunguza kiasi cha nafasi ya ziada, katika baadhi ya matukio inafaa kupanga mfuatano yote katika seti ya data kwa mpangilio wa kuongezeka kwa urefu (au, kwa usahihi zaidi, idadi ya tokeni). Hii itahakikisha kwamba kila kundi dogo lina mfuatano wa urefu unaofanana.


## Uwakilishi wa Semantiki: Word2Vec

Katika mfano wetu wa awali, safu ya embedding ilijifunza kuwakilisha maneno kwa njia ya vekta, hata hivyo, uwakilishi huo haukuwa na maana ya kisemantiki. Ingekuwa vizuri kujifunza uwakilishi wa vekta ambapo maneno yanayofanana au visawe vinahusiana na vekta zilizo karibu kwa mujibu wa umbali fulani wa vekta (kwa mfano, umbali wa euclidian).

Ili kufanikisha hilo, tunahitaji kufundisha awali mfano wetu wa embedding kwa mkusanyiko mkubwa wa maandishi kwa kutumia mbinu kama [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Mbinu hii inategemea usanifu kuu mbili zinazotumika kuzalisha uwakilishi wa maneno ulio sambamba:

 - **Mfuko endelevu wa maneno** (CBoW), ambapo tunafundisha mfano kutabiri neno kutoka muktadha wa maneno yanayozunguka. Kwa kuzingatia ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, lengo la mfano ni kutabiri $W_0$ kutoka $(W_{-2},W_{-1},W_1,W_2)$.
 - **Skip-gram endelevu** ni kinyume cha CBoW. Mfano hutumia dirisha la muktadha wa maneno yanayozunguka kutabiri neno la sasa.

CBoW ni ya haraka, na ingawa skip-gram ni polepole, inafanya kazi bora ya kuwakilisha maneno yasiyo ya kawaida.

![Picha inayoonyesha algoriti za CBoW na Skip-Gram za kubadilisha maneno kuwa vekta.](../../../../../translated_images/sw/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Ili kujaribu embedding ya Word2Vec iliyofundishwa awali kwenye seti ya data ya Google News, tunaweza kutumia maktaba ya **gensim**. Hapa chini tunapata maneno yanayofanana zaidi na 'neural'.

> **Note:** Unapounda vekta za maneno kwa mara ya kwanza, kupakua zinaweza kuchukua muda!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Tunaweza pia kutoa kiambatisho cha vector kutoka kwa neno, ili kitumike katika kufundisha modeli ya uainishaji. Kiambatisho kina sehemu 300, lakini hapa tunaonyesha tu sehemu 20 za kwanza za vector kwa uwazi:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Jambo kuu kuhusu embeddings za semantiki ni kwamba unaweza kudhibiti usimbaji wa vector kulingana na semantiki. Kwa mfano, tunaweza kuomba kupata neno ambalo uwakilishi wake wa vector uko karibu iwezekanavyo na maneno *mfalme* na *mwanamke*, na mbali iwezekanavyo na neno *mwanaume*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Mfano hapo juu unatumia uchawi wa ndani wa GenSym, lakini mantiki ya msingi ni rahisi sana. Jambo la kuvutia kuhusu embeddings ni kwamba unaweza kufanya operesheni za kawaida za vekta kwenye vekta za embedding, na hiyo itaakisi operesheni kwenye **maana** za maneno. Mfano hapo juu unaweza kuelezwa kwa muktadha wa operesheni za vekta: tunahesabu vekta inayolingana na **MFALME-MWANAMUME+MWANAMKE** (operesheni `+` na `-` zinafanywa kwenye uwakilishi wa vekta wa maneno husika), halafu tunatafuta neno lililo karibu zaidi kwenye kamusi kwa vekta hiyo:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Tulilazimika kuongeza viambatisho vidogo kwenye vekta za *man* na *woman* - jaribu kuondoa ili uone kinachotokea.

Ili kupata vekta iliyo karibu zaidi, tunatumia vifaa vya TensorFlow kuhesabu vekta ya umbali kati ya vekta yetu na vekta zote kwenye msamiati, kisha tunapata kiashiria cha neno lenye thamani ndogo kwa kutumia `argmin`.


Ingawa Word2Vec inaonekana kuwa njia nzuri ya kuelezea semantiki ya maneno, ina mapungufu mengi, ikiwa ni pamoja na yafuatayo:

* Miundo ya CBoW na skip-gram ni **upachikaji wa utabiri**, na huzingatia muktadha wa karibu tu. Word2Vec haifaidiki na muktadha wa jumla.
* Word2Vec haizingatii **mofolojia** ya maneno, yaani ukweli kwamba maana ya neno inaweza kutegemea sehemu tofauti za neno, kama mzizi wake.

**FastText** inajaribu kushinda kikwazo cha pili, na inajenga juu ya Word2Vec kwa kujifunza uwakilishi wa vekta kwa kila neno na n-gramu za herufi zinazopatikana ndani ya kila neno. Thamani za uwakilishi hizi kisha zinajumlishwa kwa wastani kuwa vekta moja katika kila hatua ya mafunzo. Ingawa hii inaongeza hesabu nyingi za ziada wakati wa mafunzo ya awali, inawezesha upachikaji wa maneno kuingiza taarifa za sehemu za maneno.

Njia nyingine, **GloVe**, inatumia mbinu tofauti kwa upachikaji wa maneno, inayotegemea ufafanuzi wa matriki ya muktadha wa maneno. Kwanza, inajenga matriki kubwa inayohesabu idadi ya matukio ya maneno katika muktadha tofauti, kisha inajaribu kuwakilisha matriki hii katika vipimo vya chini kwa njia inayopunguza hasara ya ujenzi upya.

Maktaba ya gensim inaunga mkono upachikaji wa maneno haya, na unaweza kujaribu nayo kwa kubadilisha msimbo wa kupakia modeli hapo juu.


## Kutumia embeddings zilizofundishwa awali katika Keras

Tunaweza kurekebisha mfano hapo juu ili kujaza awali matriki katika safu yetu ya embedding kwa kutumia embeddings za kisemantiki, kama Word2Vec. Msamiati wa embeddings zilizofundishwa awali na ule wa maandishi yetu huenda usifanane, kwa hivyo tunahitaji kuchagua moja. Hapa tunachunguza chaguo mbili zinazowezekana: kutumia msamiati wa tokenizer, na kutumia msamiati kutoka kwa embeddings za Word2Vec.

### Kutumia msamiati wa tokenizer

Tunapotumia msamiati wa tokenizer, baadhi ya maneno kutoka kwa msamiati yatakuwa na embeddings za Word2Vec zinazolingana, na mengine yatakosekana. Kwa kuzingatia kwamba ukubwa wa msamiati wetu ni `vocab_size`, na urefu wa vector ya embedding ya Word2Vec ni `embed_size`, safu ya embedding itawakilishwa na matriki ya uzito yenye umbo `vocab_size`$\times$`embed_size`. Tutajaza matriki hii kwa kupitia msamiati:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Kwa maneno ambayo hayapo kwenye msamiati wa Word2Vec, tunaweza kuyaacha kama sifuri, au kuzalisha vector ya nasibu.

Sasa tunaweza kufafanua safu ya embedding yenye uzito uliotayarishwa awali:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Kumbuka**: Angalia kwamba tumeweka `trainable=False` wakati wa kuunda `Embedding`, ambayo inamaanisha hatufanyi mafunzo upya ya safu ya Embedding. Hii inaweza kusababisha usahihi kuwa chini kidogo, lakini inaharakisha mchakato wa mafunzo.

### Kutumia msamiati wa embedding

Tatizo moja na mbinu ya awali ni kwamba misamiati inayotumika katika TextVectorization na Embedding ni tofauti. Ili kushinda tatizo hili, tunaweza kutumia mojawapo ya suluhisho zifuatazo:
* Kufundisha upya modeli ya Word2Vec kwa kutumia msamiati wetu.
* Kupakia seti yetu ya data kwa kutumia msamiati kutoka kwenye modeli ya Word2Vec iliyofundishwa awali. Misamiati inayotumika kupakia seti ya data inaweza kubainishwa wakati wa kupakia.

Mbinu ya pili inaonekana rahisi zaidi, kwa hivyo hebu tuitekeleze. Kwanza kabisa, tutaunda safu ya `TextVectorization` na msamiati maalum, uliotolewa kutoka kwenye embeddings za Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Maktaba ya gensim ya word embeddings ina kipengele rahisi, `get_keras_embeddings`, ambacho kitaunda kiotomatiki safu ya Keras embeddings inayolingana kwa ajili yako.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Moja ya sababu hatuoni usahihi wa juu ni kwa sababu baadhi ya maneno kutoka kwenye seti yetu ya data yanakosekana katika msamiati wa GloVe uliotangulizwa, na hivyo yanapuuzwa kimsingi. Ili kushinda hili, tunaweza kufundisha upachikaji wetu wenyewe kulingana na seti yetu ya data.


## Uwakilishi wa Muktadha wa Maneno

Kikwazo kimoja kikubwa cha uwakilishi wa maneno uliopre-sheniwa wa jadi kama Word2Vec ni kwamba, ingawa yanaweza kunasa maana fulani ya neno, hayawezi kutofautisha kati ya maana tofauti. Hili linaweza kusababisha changamoto katika mifano inayofuata.

Kwa mfano, neno 'play' lina maana tofauti katika sentensi hizi mbili:
- Nilikwenda kwenye **play** ukumbini wa maonyesho.
- John anataka **play** na marafiki zake.

Uwakilishi wa maneno uliopre-sheniwa tulioujadili unawakilisha maana zote mbili za neno 'play' kwa uwakilishi mmoja. Ili kushinda kikwazo hiki, tunahitaji kujenga uwakilishi wa maneno kulingana na **mfano wa lugha**, ambao umefundishwa kwa maandishi mengi, na *unajua* jinsi maneno yanavyoweza kuunganishwa katika muktadha tofauti. Kujadili uwakilishi wa muktadha wa maneno ni nje ya wigo wa mafunzo haya, lakini tutarudi kwenye mada hii tutakapozungumzia mifano ya lugha katika kitengo kijacho.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
